In [ ]:
# Импорт необходимых библиотек
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
from llama_index.core import Document, Settings, SimpleDirectoryReader, StorageContext, VectorStoreIndex
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import JSONLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import ElasticsearchStore
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub
from peft import AutoPeftModelForCausalLM
import torch
import torch.nn.functional as F
import os
import elasticsearch
from elasticsearch import Elasticsearch, helpers
import numpy as np
import pandas as pd
import sys
import json
from ast import literal_eval
from tqdm import tqdm
import datetime

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [ ]:
# Импорт библиотек TensorFlow и TensorFlow I/O
import tensorflow as tf
import tensorflow_io as tfio

# Импорт дополнительных модулей из TensorFlow
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [ ]:
%%capture
!pip install elasticsearch==8.8.0

In [ ]:
%%bash

rm -rf elasticsearch*
wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.8.0-linux-x86_64.tar.gz
tar -xzf elasticsearch-8.8.0-linux-x86_64.tar.gz
sudo chown -R daemon:daemon elasticsearch-8.8.0/
umount /sys/fs/cgroup
apt install cgroup-tools

Reading package lists...
Building dependency tree...
Reading state information...
cgroup-tools is already the newest version (2.0-2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


umount: /sys/fs/cgroup: not mounted.




In [ ]:
%%bash --bg

sudo -H -u daemon elasticsearch-8.8.0/bin/elasticsearch

In [ ]:
# This part is important, since it takes a little amount of time for instance to load
import time
time.sleep(120)

In [ ]:
!ps -ef | grep elastic

root       59980   59978  0 10:37 ?        00:00:00 sudo -H -u daemon elasticsearch-8.8.0/bin/elasti
daemon     59981   59980 12 10:37 ?        00:00:21 /content/elasticsearch-8.8.0/jdk/bin/java -Xms4m
daemon     60201   59981 47 10:38 ?        00:01:07 /content/elasticsearch-8.8.0/jdk/bin/java -Des.n
daemon     60269   60201  0 10:38 ?        00:00:00 /content/elasticsearch-8.8.0/modules/x-pack-ml/p
root       60833     397  0 10:40 ?        00:00:00 /bin/bash -c ps -ef | grep elastic
root       60835   60833  0 10:40 ?        00:00:00 grep elastic


In [ ]:
!/content/elasticsearch-8.8.0/bin/elasticsearch-setup-passwords auto -url "https://localhost:9200"

******************************************************************************
Note: The 'elasticsearch-setup-passwords' tool has been deprecated. This       command will be removed in a future release.
******************************************************************************

Initiating the setup of passwords for reserved users elastic,apm_system,kibana,kibana_system,logstash_system,beats_system,remote_monitoring_user.
The passwords will be randomly generated and printed to the console.
Please confirm that you would like to continue [y/N]y


Changed password for user apm_system
PASSWORD apm_system = zGS2wMJq0Jcu3V14l4zB

Changed password for user kibana_system
PASSWORD kibana_system = iem8D878tjTmU5eNRRfO

Changed password for user kibana
PASSWORD kibana = iem8D878tjTmU5eNRRfO

Changed password for user logstash_system
PASSWORD logstash_system = 07NIhk48jwwBhASVp7Gx

Changed password for user beats_system
PASSWORD beats_system = Bm2ttLe9HLQJwA4C2WKU

Changed password for user rem

In [ ]:
!curl --cacert /content/elasticsearch-8.8.0/config/certs/http_ca.crt -u elastic -H 'Content-Type: application/json' -XGET https://localhost:9200/?pretty=true


Enter host password for user 'elastic':
{
  "name" : "fb7d22aca9a8",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "sliOj35BSn-gwqDx2EJL5A",
  "version" : {
    "number" : "8.8.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "c01029875a091076ed42cdb3a41c10b1a9a5a20f",
    "build_date" : "2023-05-23T17:16:07.179039820Z",
    "build_snapshot" : false,
    "lucene_version" : "9.6.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [ ]:
username = 'elastic'

password = '7BYvOOWjEIvjV8neT7nv'

es = Elasticsearch(['https://localhost:9200'], basic_auth=(username, password), ca_certs="/content/elasticsearch-8.8.0/config/certs/http_ca.crt")

resp = dict(es.info())

resp

{'name': 'fb7d22aca9a8',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'sliOj35BSn-gwqDx2EJL5A',
 'version': {'number': '8.8.0',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': 'c01029875a091076ed42cdb3a41c10b1a9a5a20f',
  'build_date': '2023-05-23T17:16:07.179039820Z',
  'build_snapshot': False,
  'lucene_version': '9.6.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [ ]:
es.ping()

True

In [ ]:
# Путь к вашему JSON-файлу
json_file_path = '/content/ConfluencePages...json'

# Загрузка данных из JSON-файла
with open(json_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Создание списка документов
documents = [{"text": page['text'], "metadata": {
    "title": page['title'],
    "link": page['link'],
    "date": page['date'],
    "author": page['author']
}} for page in data['pages']]

# Разбиение документов на фрагменты
text_splitter = RecursiveCharacterTextSplitter(chunk_size=450, chunk_overlap=50)
fragments = text_splitter.create_documents([doc['text'] for doc in documents], [doc['metadata'] for doc in documents])

In [ ]:
# Загрузка модели для векторных представлений
embed_model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.69k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [ ]:
# Загрузка модели для генерации ответов
adapt_model_name = "IlyaGusev/saiga_mistral_7b_lora"
base_model_name = "Open-Orca/Mistral-7B-OpenOrca"
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
device_map = {"": "cpu"}

model = AutoPeftModelForCausalLM.from_pretrained(adapt_model_name, device_map=device_map, torch_dtype=torch.bfloat16)

tokenizer_config.json:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/101 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


adapter_model.safetensors:   0%|          | 0.00/54.6M [00:00<?, ?B/s]

In [ ]:
# Функция для получения эмбеддингов
def get_embedding(sentence):
    return embed_model.encode(sentence, convert_to_tensor=True)

In [ ]:
# Вычисление эмбеддингов для всех фрагментов
embeddings = [get_embedding(fragment.page_content) for fragment in fragments]

In [ ]:
# Проверяем, существует ли индекс "articles" в Elasticsearch
if not es.indices.exists(index="articles"):
    try:
        # Определяем настройки и маппинг для нового индекса
        es_index = {
            "settings": {
                "index": {
                    "number_of_shards": 1,  # Устанавливаем количество шардов (разделов) индекса
                    "number_of_replicas": 0  # Устанавливаем количество реплик (копий) индекса
                }
            },
            "mappings": {
                "properties": {
                    "text": {"type": "text"},  # Поле для текстового контента
                    "link": {"type": "keyword"},  # Поле для хранения ссылок (необрабатываемый текст)
                    "embedding": {"type": "dense_vector", "dims": 512}  # Поле для хранения векторов эмбеддингов размером 512
                }
            }
        }

        # Создаем индекс "articles" с заданными настройками и маппингом
        es.indices.create(index="articles", body=es_index)
        print("Индекс 'articles' создан")  # Выводим сообщение об успешном создании индекса
    except Exception as e:
        # Обрабатываем исключения, если создание индекса завершилось неудачей
        print(f"Ошибка при создании индекса 'articles': {e}")  # Выводим сообщение об ошибке
else:
    # Если индекс уже существует, выводим соответствующее сообщение
    print("Индекс 'articles' уже существует")


<ipython-input-20-bf2b36945acb>:22: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index="articles", body=es_index)
<ipython-input-20-bf2b36945acb>:25: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index="articles", body={


In [ ]:
# Функция для поиска статей в Elasticsearch с использованием векторных представлений
def search_articles(query, top_k=5):
    query_embedding = get_embedding(query).tolist()  # Получаем векторное представление запроса

    script_query = {
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                "params": {"query_vector": query_embedding}
            }
        }
    }

    response = es.search(index="articles", body={
        "size": top_k,
        "query": script_query,
        "_source": ["text", "link"]
    })

    return response['hits']['hits']

In [ ]:
# Определение устройства (GPU, если доступен, иначе CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def generate_response(question, context):
    # Объединяем вопрос и контекст и подготавливаем их для токенизации
    inputs = tokenizer(question + context, return_tensors="pt", padding=True, truncation=True)

    # Переносим все входные данные на заданное устройство (например, GPU или CPU)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Отключаем градиенты для экономии памяти и ускорения вычислений
    with torch.no_grad():
        # Генерируем ответ с помощью модели
        outputs = model.generate(
            input_ids=inputs["input_ids"],  # Входные идентификаторы токенов
            attention_mask=inputs["attention_mask"],  # Маска внимания для обработки входных данных
            max_new_tokens=150,  # Максимальное количество генерируемых токенов
            num_return_sequences=1,  # Количество возвращаемых последовательностей (ответов)
            pad_token_id=tokenizer.eos_token_id  # Идентификатор токена для заполнения (обычно конец последовательности)
        )

    # Декодируем сгенерированный ответ в текстовый формат, пропуская специальные токены
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Возвращаем сгенерированный ответ
    return response

In [ ]:
from langchain.document_transformers import LongContextReorder

# Создаем экземпляр класса LongContextReorder для переупорядочивания контекста
reorderer = LongContextReorder()

def answer_question_with_reorder(query, reorder=True, print_results=False):
    # Выполняем поиск статей по заданному запросу
    results = search_articles(query)

    # Если требуется, выводим результаты поиска
    if print_results:
        for hit in results:
            print(f"{hit['_source']['text']}\n--------")

    # Если переупорядочивание включено, применяем его к результатам
    if reorder:
        results = reorderer.transform_documents(results)

    # Объединяем текст из всех результатов в один контекст
    context = " ".join([hit["_source"]["text"] for hit in results])

    # Генерируем ответ на основе запроса и контекста
    response = generate_response(query, context)

    # Предполагаем, что первый результат содержит наиболее релевантную ссылку
    link = results[0]["_source"]["link"] if results else None

    # Возвращаем сгенерированный ответ и ссылку на источник
    return {
        "response": response,
        "link": link
    }

def interactive_search_with_reorder():
    # Бесконечный цикл для интерактивного ввода вопросов
    while True:
        inp_question = input("Пожалуйста, введите вопрос: ")  # Запрашиваем ввод у пользователя
        result = answer_question_with_reorder(inp_question, reorder=True, print_results=True)  # Получаем ответ
        print("\nОтвет от модели SaigaMistral:")
        print(result["response"])  # Выводим ответ от модели

        # Если ссылка на источник доступна, выводим её
        if result["link"]:
            print(f"\nСсылка на источник: {result['link']}")

        print("\n\n========\n")  # Разделитель для удобства чтения

# Основная функция, запускающая интерактивный поиск
if __name__ == "__main__":
    interactive_search_with_reorder()

In [ ]:
# Список вопросов для тестирования
test_questions = [
    "Как быть если заказчик требует водителя с мед книжкой?",
    "Что входит в доставку в гипермаркеты?",
    "Как занести контакт в черный список?",
    "Доступна ли перевозка грузов в Киргизию?",
    "Как найти груз по номеру интернет заказа?",
    "Как изменить страховую сумму при оформлении груза?",
    "Возможна ли доставка день в день? Какие интервалы доставки?",
    "Что делать, если клиент предоставил новый номер для оповещения?",
    "Почему не выходят на печать чеки об оплате и ПКО не отображается в реестре?",
    "Как клиент может изменить дату авизации в личном кабинете? ",
    "Клиент спрашивает, для чего мы запрашиваем письма от клиента, при предоставлении ему очередного спец условия",
    "Что написать клиенту, который просит полный доступ к личному кабинету?",
    "Какая упаковка используется при перевозке автомобильных стекол?",
    "Сколько времени может проработать подметальная машина?",
    "Что делать, если клиент отказывается получения сопроводительных документов?",
    "Какие есть размеры паллетных рам?",
    "Как осуществить Контроль выполнения НТМЦ?",
    "При отправке грузов в Минск прямые или транзитные рейсы?",
    "Какие документы нужны для выдачи грузов в Беларусь (для физ. и юр. лиц)?",
    "Что делать, если клиент выбрал способ подписания Договорных документов на бумаге?",
    "Как провести обработку упрощенной выдачи по нескольким грузам?",
    "Когда следует использовать нестандартную защитную упаковку?",
    "Как следует грузить негабаритный груз?",
    "Что включает общий вид карточки контрагента?",
    "Как загрузить прайс лист по услугам FTL?",
    "Что включает отчет по агентским вознаграждениям FTL?",
    "Как поступить, если у заявки выставлен статус 'Подтверждение заявки с клиентом'",
    "Как создать новую географическую зону?",
    "Какие бизнес правила страхования актуальны сегодня?",
    "Кто изменял документ РасчитатьСпецУсловие_ЗаменаУслуг",
    "Кто наполняет таблицу в базе данных api.RawLog",
    "Когда создана форма документа споВосстановлениеМДИ",
    "Как реализован механизм многопоточной обработки ЭТН?",
    "Кто на форму Обработка.ЗачетПереплат добавлял элементы ПериодВыставленияАктов и КонтрагентыНачинаютсяССимволов",
    "Что произойдет с просроченными грузами в ПВЗ?",
    "Возможно ли создание большего количества характеров груза, чем в документе  'Настройка матрица соответствия внутреннего груза'",
    "Выставление документа Счет на оплату клиенту, при изменении данных о плательщики, городе оплаты и сумме.",
    "Возможно ли изменить возвратные данные по грузу, до того, как груз станет возвратным?",
    "Была ли доработка документа 'Инвентаризация на отправку'?",
    "Была ли доработка документа 'Приемка'?"

]

In [ ]:
def test_questions_with_reorder():
    # Проходим по каждому вопросу в списке тестовых вопросов
    for question in test_questions:
        print(f"Вопрос: {question}")  # Выводим текущий вопрос

        # Получаем ответ на вопрос с переупорядочиванием контекста
        result = answer_question_with_reorder(question, reorder=True, print_results=False)

        print("\nОтвет от модели SaigaMistral:")  # Заголовок для ответа
        print(result["response"])  # Выводим ответ от модели

        # Если ссылка на источник доступна, выводим её
        if result["link"]:
            print(f"\nСсылка на источник: {result['link']}")

        print("\n\n========\n")  # Разделитель для удобства чтения

# Основная функция, запускающая тестирование вопросов
if __name__ == "__main__":
    test_questions_with_reorder()
